In [1]:
import urllib.request as urlrequest
from pathlib import Path

base_data_path = Path() / "data" 

base_data_path.mkdir(parents = True, exist_ok = True)

zip_path = base_data_path / "dataset.zip"

if not zip_path.exists():

    dataset_url = "https://www.kaggle.com/api/v1/datasets/download/alvarole/asirra-cats-vs-dogs-object-detection-dataset"


    data_path = Path() / "data" / "dataset.zip"

    response = urlrequest.urlopen(
        dataset_url,
    )

    download_size = response.getheader("Content-Length")
    with open(data_path, "wb") as f:

        f.write(response.read())


In [2]:

import zipfile
import os

def unzip():

    inner_file = "Asirra_ cat vs dogs"
    with zipfile.ZipFile(zip_path, "r") as zip:
        
        for item in zip.infolist():

            zip.extract(item, base_data_path)

    dataset_location = base_data_path / "dataset"
    os.rename(base_data_path / inner_file, dataset_location)

# unzip()


In [14]:

import itertools
import xml.etree.ElementTree as ET


dataset_location = base_data_path / "dataset"
def patched_dataset_paths(dataset_location):

    return itertools.batched(dataset_location.iterdir(), 2)


# specific xml reader implementation for the lolz
def read_metadata(xml_file: Path):

    with open(xml_file, "r", encoding = "utf-8") as f:
        text = ET.canonicalize(from_file=f, strip_text = True)
        
    tree = ET.fromstring(text)

    size = tree.find("size")
    size = {elem.tag: int(elem.text) for elem in size.iter() if not elem.tag == "size"}


    objects = tree.findall("object")
    objects = [dict(
        name = obj.find("name").text,
        pose = obj.find("pose").text,
        truncated = int(obj.find("truncated").text),
        difficult = int(obj.find("difficult").text),
        bndbox = {
            elem.tag: float(elem.text) 
            for elem in obj.find("bndbox").iter()
            if not elem.tag == "bndbox"
        }
    ) for obj in objects]

    metadata = dict(
        size = size,
        objects = objects
    )

    return metadata

meta = []
invalids = []
for img, xml_path in patched_dataset_paths(dataset_location):

    metadata = read_metadata(xml_path) | dict(img_path = img)
    meta.append(metadata)

print(meta[-1])

{'size': {'width': 500, 'height': 374, 'depth': 3}, 'objects': [{'name': 'dog', 'pose': 'Unspecified', 'truncated': 0, 'difficult': 0, 'bndbox': {'xmin': 58.0, 'ymin': 16.0, 'xmax': 469.0, 'ymax': 373.0}}], 'img_path': WindowsPath('data/dataset/dog.99.jpg')}
